In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import PyPDFLoader, TextLoader, UnstructuredFileLoader
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
    CharacterTextSplitter,
)

In [2]:
# vectorization
# 각각의 문서마다 벡터를 만들어줄 거 -> embed
# openAI의 embed 모델은 최소 1000차원을 갖는 벡터를 제공함

# 그럼 무엇을 embed하는 거냐? -> 단어를 embed하게 됨